# Creación del DataFrame df_U_FR

1. [Carga del DataFrame](#carga-del-dataframe)
<br></br>
2. [Localización del Producto Estrella Teniendo en Cuenta el País](#localizacion-del-producto-estrella-teniendo-en-cuenta-el-pais)
<br></br>
3. [Creación del DataFrame df_U_FR](#creacion-del-dataframe-df_U_FR)
<br></br>
4. [Creación del Archivo con Formato Pickle del DataFrame df_U_FR](#creacion-del-archivo-con-formato-pickle-del-dataframe-df_U_FR)
<br></br>
5. [Conclusiones](#conclusiones)

In [1]:
%load_ext watermark
%watermark

2019-06-14T14:01:05+02:00

CPython 3.6.8
IPython 6.2.1

compiler   : GCC 7.3.0
system     : Linux
release    : 4.13.0-46-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 2
interpreter: 64bit


## 1. <a id='carga-del-dataframe'>Carga del DataFrame</a>

In [2]:
import pandas as pd
import datetime
import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_pickle('../Data/01.-Exploracion-de-los-Datos-y-Creacion-DataFrame-Base.pkl')

df.head()

,Fecha_envio,Product_code,Origen,Destino,Peso,Volumen,Peso_final
0,2017-05-01,B,ES,NO,1.80,3.584,3.584
1,2017-05-01,B,ES,NO,0.90,2.242,2.242
2,2017-05-01,B,ES,NO,0.35,1.009,1.009
3,2017-05-01,B,ES,NO,1.15,2.130,2.130
4,2017-05-01,B,ES,NO,0.45,0.996,0.996


## 2. <a id='localizacion-del-producto-estrella-teniendo-en-cuenta-el-pais'>Localización del Producto Estrella Teniendo en Cuenta el País</a>

In [4]:
df['Product_code'].unique()

array(['B', 'U', 'P', 'D', 'N', 'C', 'T', 'W', 'H', 'Q', '7', '9', '8',
       '1', 'X', 'K', 'V', 'Y', 'I', 'L', 'M', 'E'], dtype=object)

In [5]:
df_shippings = df.groupby(['Destino','Product_code']).size().reset_index()

df_shippings.head()

,Destino,Product_code,0
0,AD,7,192
1,AD,8,16
2,AD,D,7148
3,AD,H,3
4,AD,N,4


In [6]:
df_shippings.columns = ['Destino','Producto','Total']

df_shippings = df_shippings.sort_values(by = 'Total',ascending = False)

df_shippings.head()

,Destino,Producto,Total
422,ES,N,1163851
470,FR,U,1157568
705,IT,U,974392
345,DE,U,771554
495,GB,U,717257


El 'Producto' N es del que más envíos se realiza, teniendo en cuenta la segmentación de países. Sin embargo es el producto **U para FR** el que vamos a considerar, pues:
* Se realizan prácticamente los mismos envíos que de ES N.
* El margen de ingreso por envío es considerablemente mayor.

In [7]:
df_shippings_FR = df_shippings[df_shippings['Destino'] == 'FR']

df_shippings_FR.head()

,Destino,Producto,Total
470,FR,U,1157568
471,FR,W,71554
469,FR,T,41463
458,FR,7,5817
462,FR,C,2796


In [8]:
df_shippings_FR['Porcentaje_del_total'] = ((df_shippings_FR['Total'] / df_shippings_FR['Total'].sum()) * 100).round(2)

df_shippings_FR.head()

,Destino,Producto,Total,Porcentaje_del_total
470,FR,U,1157568,90.13
471,FR,W,71554,5.57
469,FR,T,41463,3.23
458,FR,7,5817,0.45
462,FR,C,2796,0.22


El producto U para Francia representa un 90.13% de los envíos, lo cual refuerza la decisión de considerarlo el producto estrella teniendo en cuenta la segmentación por países.

Vamos a crear un DataFrame únicamente para los campos 'Fecha_envio' y 'Total', para el 'Producto' U en el 'Destino' FR.

## 3. <a id='creacion-del-dataframe-df_U_FR'>Creación del DataFrame df_U_FR</a>

In [9]:
df.head()

,Fecha_envio,Product_code,Origen,Destino,Peso,Volumen,Peso_final
0,2017-05-01,B,ES,NO,1.80,3.584,3.584
1,2017-05-01,B,ES,NO,0.90,2.242,2.242
2,2017-05-01,B,ES,NO,0.35,1.009,1.009
3,2017-05-01,B,ES,NO,1.15,2.130,2.130
4,2017-05-01,B,ES,NO,0.45,0.996,0.996


In [10]:
df_U_FR = df[(df['Product_code'] == 'U') & (df['Destino'] == 'FR')]

df_U_FR.head()

,Fecha_envio,Product_code,Origen,Destino,Peso,Volumen,Peso_final
27,2017-05-01,U,ES,FR,0.15,0.606,0.606
30,2017-05-01,U,ES,FR,2.85,7.449,7.449
31,2017-05-01,U,ES,FR,0.65,1.949,1.949
48,2017-05-01,U,ES,FR,0.35,1.071,1.071
49,2017-05-01,U,ES,FR,1.05,3.601,3.601


In [11]:
df_U_FR = df_U_FR.groupby(['Fecha_envio']).size().reset_index()

df_U_FR.head()

,Fecha_envio,0
0,2017-05-01,48
1,2017-05-02,2371
2,2017-05-03,2640
3,2017-05-04,2565
4,2017-05-05,1717


Aprovechamos para renombrar las columnas como nos exige Prophet para procesar el dataframe.

In [12]:
df_U_FR.columns = ['ds','y']

df_U_FR.head()

,ds,y
0,2017-05-01,48
1,2017-05-02,2371
2,2017-05-03,2640
3,2017-05-04,2565
4,2017-05-05,1717


In [13]:
df_U_FR.tail()

,ds,y
648,2019-03-27,2346
649,2019-03-28,2029
650,2019-03-29,1880
651,2019-03-30,14
652,2019-03-31,93


## 4. <a id='creacion-del-archivo-con-formato-pickle-del-dataframe-df_U_FR'>Creación del Archivo con Formato Pickle del DataFrame df_U_FR</a>

In [14]:
df_U_FR.to_pickle('../Data/02.-Creacion-del-DataFrame-df_U_FR.pkl')

In [15]:
!ls -l --block-size=M ../Data/02.-Creacion-del-DataFrame-df_U_FR.pkl

-rw-r--r-- 1 dsc dsc 1M jun 14 14:01 ../Data/02.-Creacion-del-DataFrame-df_U_FR.pkl


## 5. <a id='conclusiones'>Conclusiones</a>

* El producto estrella es U en Francia, que además representa más del 90% de los envíos en este país. Y se ha creado el dataframe que muestra su comportamiento en el rango de fechas 05/2017 hasta 03/2019, para posteriormente procesarlo con Prophet.

* El dataframe df_U_FR ha sido guardado en el archivo '02.-Creacion-del-DataFrame-df_U_FR.pkl'.